In [16]:
from pyspark import SparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
emotions=sc.textFile("s3://ziqiproject/input/*.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
def solve_comma(s):
    """We have two situation, if there is " in the beginning, we need to find the last " to make sure that is the text, otherwise just use comma to split"""
    if s[0]=='"':
        return (s[:s.rindex('"')],s[(s.rindex('"')+2):].split(','))
    words = s.split(',')
    return (words[0],words[1:])
emotion_1 = emotions.map(lambda s:solve_comma(s))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
#Eliminate the header and non-clear emotion example with no emotion
emotion_2 = emotion_1.filter(lambda s:s[0]!='text' and s[1][7]=='False')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
#Eliminate useless columns, maintain text and corresponding emotion list
emotion_3 = emotion_2.map(lambda s:(s[0],s[1][8:]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [115]:
#Get the emotion list
emotion_list = emotion_1.take(1)[0][1][8:]
emotion_list

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']

In [128]:
def emotion_text(s):
    """Check which column has value of 1 and write the emotion name on the corresponding emotion list"""
    string = ''
    for i in range(len(s[1])):
        if s[1][i]=='1':
            string+=emotion_list[i]+','
    if len(string)>0:
        string = string[:len(string)-1]
    return (s[0],string)
emotion_4 = emotion_3.map(lambda s:emotion_text(s))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
# Transfer it to be a json file
result = emotion_4.map(lambda s:{'text':s[0],'emotion':s[1]})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
result.saveAsTextFile("s3://ziqiproject/output/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…